In [7]:
import pandas as pd
from textblob import TextBlob
from nltk.tokenize.toktok import ToktokTokenizer
import re
tokenizer = ToktokTokenizer()
import spacy
nlp = spacy.load('en_core_web_sm', disable=['ner'])

In [8]:
import csv
train = pd.read_csv("Train.csv")
train

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


In [9]:
label_0=train[train['label']==0].sample(n=5000)
label_1=train[train['label']==1].sample(n=5000)

In [10]:
train=pd.concat([label_1,label_0])
from sklearn.utils import shuffle
train = shuffle(train)

In [11]:
train.isnull().sum()

text     0
label    0
dtype: int64

In [12]:
import numpy as np
train.replace('r^\s*', np.nan, regex=True, inplace=True)
train.dropna(axis = 0, how = 'any', inplace=True)

In [13]:
train.replace(to_replace=["r\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
print('escape seq removed')

escape seq removed


In [14]:
import numpy as np
train.replace('r^\s*$', np.nan, regex=True,inplace=True)
train.dropna(axis = 0, how = 'any', inplace = True)

In [15]:
train['text']=train['text'].str.encode('ascii','ignore').str.decode('ascii')
print('non-ascii data removed')

non-ascii data removed


In [16]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [17]:
def remove_punctuations(text):
  import string
  for punctuation in string.punctuation:
    text = text.replace(punctuation, '')
    return text
    train['text']=train['text'].apply(remove_punctuations)

In [18]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

In [20]:
def custom_remove_stopwords(text, is_lower_case=False):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens] 
  if is_lower_case:
    filtered_tokens = [token for token in tokens if token not in stopword_list]
  else:
     filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
     filtered_text = ''.join(filtered_tokens)
  return filtered_text

In [21]:
train['text']=train['text'].apply(custom_remove_stopwords)

In [22]:
def remove_special_characters(text):
  text = re.sub('[^a-zA-z0-9\s]', '', text)
  return text

In [23]:
train['text']=train['text'].apply(remove_special_characters)

In [24]:
def remove_html(text):
  import re
  html_patter = re.compile('<.*?>')
  return html_patter.sub(r'',text)

In [25]:
train['text']=train['text'].apply(remove_html)

In [26]:
def remove_numbers(text):
  """ Remove integers """
  text = ''.join([i for i in text if not i.isdigit()])
  return text

In [27]:
train['text']=train['text'].apply(remove_numbers)

In [28]:
def lemmatize_text(text):
  text = nlp(text)
  text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
  return text

In [29]:
train['text']=train['text'].apply(lemmatize_text)

In [30]:
train

,text,label
22391,dominatingconflictcouplefineactorshuppertdutro...,0
16789,sawfilmfestivalyearsagoimpressedstartedlooking...,1
352,prettylandscapeshotsWritersputtingtritemouthin...,0
35251,actuallytrawledentiresetreviewssearchingonesga...,0
20142,moviewrittendirectedproducedintentioncreatingb...,0
...,...,...
15286,minutescruelloveoldMunstersYesplotthingyesline...,0
11262,onewarnerbrothersbesthighestgrossingfilmsthirt...,1
38195,earlycoppolasublimecastfolksnevergotseepitywon...,1
39426,MrAccidentdeliberateseriesnonstopdisastersnear...,0


In [56]:
sentiment_series = train['sentiment'].tolist()

In [62]:
train.loc[train['label']==0, 'sentiment'] = "Negative"
train.loc[train['label']==1, 'sentiment'] = "Positive"

In [63]:
train

,text,label,sentiment
22391,dominatingconflictcouplefineactorshuppertdutro...,0,Negative
16789,sawfilmfestivalyearsagoimpressedstartedlooking...,1,Positive
352,prettylandscapeshotsWritersputtingtritemouthin...,0,Negative
35251,actuallytrawledentiresetreviewssearchingonesga...,0,Negative
20142,moviewrittendirectedproducedintentioncreatingb...,0,Negative
...,...,...,...
15286,minutescruelloveoldMunstersYesplotthingyesline...,0,Negative
11262,onewarnerbrothersbesthighestgrossingfilmsthirt...,1,Positive
38195,earlycoppolasublimecastfolksnevergotseepitywon...,1,Positive
39426,MrAccidentdeliberateseriesnonstopdisastersnear...,0,Negative
